In [1]:
import json
import pymysql as mc
import sqlalchemy
from sqlalchemy import create_engine
from getpass import getpass
# import Credentials as cred
import pandas as pd

In [2]:
file1 = 'data/yelp_academic_dataset_business.json'
file2 = 'data/yelp_academic_dataset_checkin.json'
file3 = 'data/yelp_academic_dataset_photo.json'
file4 = 'data/yelp_academic_dataset_review.json'
file5 = 'data/yelp_academic_dataset_tip.json'
file6 = 'data/yelp_academic_dataset_user.json'
files = [file1,file2,file3,file4,file5,file6]
names = ['business','checkin','photo','review','tip','user']

In [3]:
pw = getpass()

········


In [4]:
cnx = create_engine('mysql+pymysql://'+'root'+':'+pw+'@'+'localhost'+':3306/'+'yelp',echo=False)

In [5]:
for i, file in enumerate(files):
    # don't care about checkins
    if names[i] in ['business', 'checkin', 'photo', 'review', 'tip']:
        continue
    
    # read the entire file into a python array
    with open(file, 'r') as f:
        data = f.readlines()
        data = [x.encode('utf-8').decode('latin-1') for x in data]
    
    print (f'done reading the lines of {names[i]}')
    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)
    data_json_str = "[" + ','.join(data) + "]"
    data = None
    # now, load it into pandas
    data_df = pd.read_json(data_json_str)
    data_json_str = None 
    
    print (f'done getting the dataframe of {names[i]}')
    if names[i] == 'business':
        data_df = data_df.drop(columns=['attributes','hours'])
    if names[i] == 'user':
        data_df = data_df.drop(columns=['friends'])
    #data_df.to_csv(names[i] + '.csv', index = False)
    data_df.to_sql(names[i],cnx,if_exists='replace', index=False)
    print (f'done with {names[i]}')
    data_df = None

done reading the lines of user
done getting the dataframe of user


/Users/edgarpal/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


done with user


In [13]:
# Run this in SQL

## Photo
# ALTER TABLE photo modify photo_id VARCHAR(22);
# ALTER TABLE photo ADD PRIMARY KEY (photo_id);

## Tip
# no primary key

## Business
# ALTER TABLE business modify business_id VARCHAR(22);
# ALTER TABLE business ADD PRIMARY KEY (business_id);

## Review
# ALTER TABLE review modify review_id VARCHAR(22);
# ALTER TABLE review modify user_id VARCHAR(22);
# ALTER TABLE review ADD PRIMARY KEY (review_id);

## User
# ALTER TABLE user modify user_id VARCHAR(22);
# ALTER TABLE user ADD PRIMARY KEY (user_id);

In [19]:
temp_df = data_df[['average_stars','name','review_count','useful','user_id','yelping_since','fans']]

In [21]:
temp_df.to_sql('user',cnx,if_exists='append',index=False)